In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import re

In [68]:
headers={"authority": "www.amazon.com",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-language": "en-US;q=0.9,bn;q=0.8", "sec-ch-ua":'" Not A;Brand";v="99","Chromium";v="102", "Google Chrome";v="102"'
}
url="https://www.amazon.com/Heat-Storm-HS-1500-PHX-WIFI-Infrared-Heater/product-reviews/B07JXRWJ8D/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
len_page=100

In [69]:
def reviewsHtml(url,len_page):
    soups=[]
    for page_no in range(1,len_page+1):
        params={
            'ie':'UTF8',
            'reviewerType':'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no
        }
        response=requests.get(url,headers=headers)
        soup=BeautifulSoup(response.text,'lxml')
        soups.append(soup)
    return soups

In [70]:
def getReviews(html_data):
    data_dicts=[]
    boxes=html_data.select('div[data-hook="review"]')
    for box in boxes:
        try:
            name=box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name='N/A'
        try:
            stars=box.select_one('[data-hook="review-star-rating"]').text.strip().split('out')[0]
        except Exception as e:
            stars='N/A'
        try:
            title=box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title='N/A'        

        try:
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip()
            
            match = re.search(r"on ([\w ,]+)", datetime_str)
#             print('Unparsed ',datetime_str,'\n','Match ',match)
            try:
                date = parse(match[1]).strftime("%d/%m/%Y")
            except Exception as e:
                date = 'N/A'
#             print('Parsed ',date,'\n')
        except Exception as e:
            date = 'N/A'
        
        try:
            description=box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description='N/A'
        data_dict={'Name':name,
                  'Stars':stars,
                  'Title':title,
                  'Date':date,
                  'Description':description}
        data_dicts.append(data_dict)
    return data_dicts

In [75]:
html_datas=reviewsHtml(url,len_page)
reviews=[]
for html_data in html_datas:
    review=getReviews(html_data)
    reviews +=review

In [76]:
df_reviews=pd.DataFrame(reviews)

In [77]:
len(df_reviews)

1000

In [78]:
# print('Title: ', df_reviews['Title'][53] ,'\n\nDescription',df_reviews['Description'][53])
print(df_reviews.iloc[530])

Name                                                   SlimThick
Stars                                                       5.0 
Title                       5.0 out of 5 stars\nThese r the best
Date                                                  03/04/2024
Description    We have a 3br 1bath home w large living & kitc...
Name: 530, dtype: object


In [79]:
df_reviews.to_csv(r"D:\Macys\Web Scrapping\Sample_02.csv",index=False)
# Took nearly 3 mins

In [81]:
df_reviews.isna().sum()

Name           0
Stars          0
Title          0
Date           0
Description    0
dtype: int64

In [83]:
len(df_reviews[df_reviews['Name']=='NA'])
# So there are 100 Names that are NA but these are user inputed and not real NAs

100

In [87]:
len(df_reviews[df_reviews['Description']=='NA'])

0